<a href="https://colab.research.google.com/github/yuliaplaksina/Machine-learning/blob/master/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Часть 1
## Exploratory data analysis with Pandas

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
import io
from google.colab import files
uploaded = files.upload()

Saving adult.data.csv to adult.data.csv


In [3]:
data = pd.read_csv('adult.data.csv', sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
#Выведем количество мужчин/женщин
data["sex"].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

In [5]:
#Средний возраст мужчин
data[data["sex"] == "Male"]["age"].mean()

39.43354749885268

In [6]:
#Процент жителей Германии
float((data['native-country'] == 'Germany').sum()) / data.shape[0]

0.004207487485028101

In [7]:
#Среднее значение и стандартное отклонение тех, кто зарабатывает больше 50 тысяч и меньше 50 тысяч в год

ages1 = data.loc[data['salary'] == '>50K', 'age']
ages2 = data.loc[data['salary'] == '<=50K', 'age']
print("The average age of the rich: {0} +- {1} years, poor - {2} +- {3} years.".format(
    round(ages1.mean()), round(ages1.std(), 1),
    round(ages2.mean()), round(ages2.std(), 1)))

The average age of the rich: 44 +- 10.5 years, poor - 37 +- 14.0 years.


In [8]:
#Все ли люди, получающие более 50 тысяч в год, имеют высшее образование
if len(data.loc[data['salary'] == '>50K', 'education'].unique()) > 0:
    print("Ложь")
else:
    print("Правда")

Ложь


In [9]:
#Отображение статистики возраста для каждой расы и каждого пола
data.groupby(["race", "sex"])["age"].describe()

count       mean        std   min   25%   50%  \
race               sex                                                       
Amer-Indian-Eskimo Female    119.0  37.117647  13.114991  17.0  27.0  36.0   
                   Male      192.0  37.208333  12.049563  17.0  28.0  35.0   
Asian-Pac-Islander Female    346.0  35.089595  12.300845  17.0  25.0  33.0   
                   Male      693.0  39.073593  12.883944  18.0  29.0  37.0   
Black              Female   1555.0  37.854019  12.637197  17.0  28.0  37.0   
                   Male     1569.0  37.682600  12.882612  17.0  27.0  36.0   
Other              Female    109.0  31.678899  11.631599  17.0  23.0  29.0   
                   Male      162.0  34.654321  11.355531  17.0  26.0  32.0   
White              Female   8642.0  36.811618  14.329093  17.0  25.0  35.0   
                   Male    19174.0  39.652498  13.436029  17.0  29.0  38.0   

                             75%   max  
race               sex                  
Amer-Indian-Eskimo Female  46.00  80.0  
                   Male    45.00  82.0  
Asian-Pac-Islander Female  43.75  75.0  
                   Male    46.00  90.0  
Black              Female  46.00  90.0  
                   Male    46.00  90.0  
Other              Female  39.00  74.0  
                   Male    42.00  77.0  
White              Female  46.00  90.0  
                   Male    49.00  90.0

In [10]:
#Максимальный возраст мужчин американо-индийско-эскимосской расы
data[(data["race"] == "Amer-Indian-Eskimo") & (data["sex"] == "Male")]["age"].max()

82

In [11]:
#Среди кого больше доля тех, кто много зарабатывает (>50 тыс.): женатые или одинокие мужчины (особенность семейного положения)

print("Холостяки:")
data.loc[(data['sex'] == 'Male') &
     (data['marital-status'].isin(['Never-married', 
                                   'Separated', 
                                   'Divorced',
                                   'Widowed'])), 'salary'].value_counts()

Холостяки:


<=50K    7552
>50K      697
Name: salary, dtype: int64

In [12]:
print("Женатые:")
data.loc[(data['sex'] == 'Male') & (data['marital-status'].str.startswith('Married')), 'salary'].value_counts()

Женатые:


<=50K    7576
>50K     5965
Name: salary, dtype: int64

In [13]:
# Максимальное количество часов, которые люди работают в неделю
# Сколько людей работает такое количество часов и каков процент тех, кто зарабатывает много среди них?

max_load = data['hours-per-week'].max()
print("Max time - {0} hours./week.".format(max_load))

num_workaholics = data[data['hours-per-week'] == max_load].shape[0]
print("Total number of such hard workers {0}".format(num_workaholics))

rich_share = float(data[(data['hours-per-week'] == max_load) & (data['salary'] == '>50K')].shape[0]) / num_workaholics
print("Percentage of rich (>50K) among them {0}%".format(int(100 * rich_share)))

Max time - 99 hours./week.
Total number of such hard workers 85
Percentage of rich (>50K) among them 29%


In [14]:
# Среднее время работы (hours-per-week) для тех, кто зарабатывает мало и много (salary) для каждой страны (native-country).
pd.crosstab(data['native-country'], data['salary'], 
           values=data['hours-per-week'], aggfunc=np.mean).T

native-country,?,Cambodia,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,France,Germany,Greece,Guatemala,Haiti,Holand-Netherlands,Honduras,Hong,Hungary,India,Iran,Ireland,Italy,Jamaica,Japan,Laos,Mexico,Nicaragua,Outlying-US(Guam-USVI-etc),Peru,Philippines,Poland,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
salary,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
<=50K,40.164760,41.416667,37.914634,37.381818,38.684211,37.985714,42.338235,38.041667,36.030928,40.483333,41.058824,39.139785,41.809524,39.360656,36.325,40.0,34.333333,39.142857,31.3,38.233333,41.44,40.947368,39.625,38.239437,41.000000,40.375,40.003279,36.09375,41.857143,35.068966,38.065693,38.166667,41.939394,38.470588,39.444444,40.15625,33.774194,42.866667,37.058824,38.799127,37.193548,41.6
>50K,45.547945,40.000000,45.641026,38.900000,50.000000,42.440000,47.000000,48.750000,45.000000,44.533333,50.750000,44.977273,50.625000,36.666667,42.750,NaN,60.000000,45.000000,50.0,46.475000,47.50,48.000000,45.400,41.100000,47.958333,40.000,46.575758,37.50000,NaN,40.000000,43.032787,39.000000,41.500000,39.416667,46.666667,51.43750,46.800000,58.333333,40.000000,45.505369,39.200000,49.5


# Часть 2

In [15]:
!python -m pip install --upgrade pip
!pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.1.1)


In [16]:
from google.colab import files
user_usage = files.upload()

Saving user_usage.csv to user_usage.csv


In [17]:
user_usage = pd.read_csv('user_usage.csv', sep=',')
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [18]:
user_device = files.upload()

Saving user_device.csv to user_device.csv


In [19]:
user_device = pd.read_csv('user_device.csv', sep=',')
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [20]:
devices = files.upload()

Saving android_devices.csv to android_devices.csv


In [21]:
devices = pd.read_csv('android_devices.csv', sep=',')
devices.head()

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


In [22]:
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id')
result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


Второй вариант использования merge представлен ниже, при этом таблица не сохраняется, поэтому такой вариант можно использовать только если в дальнейшем нет необходимости использовать результатирующую таблицу

In [23]:
user_usage.merge(user_device[["use_id", "platform", "device"]], on="use_id").head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [24]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


Сравним, какое из способов работает быстрее.

In [25]:
%%timeit
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id')
result.head()

100 loops, best of 3: 4.02 ms per loop


In [26]:
%%timeit
user_usage.merge(user_device[["use_id", "platform", "device"]], on="use_id").head()

100 loops, best of 3: 4.01 ms per loop


In [27]:
%%timeit
pysqldf("""SELECT us.outgoing_mins_per_month, us.outgoing_sms_per_month, us.monthly_mb, dev.use_id, dev.platform, dev.device
           FROM user_usage AS us JOIN user_device AS dev
           ON us.use_id = dev.use_id""").head()

100 loops, best of 3: 13.8 ms per loop


In [28]:
devices.rename(columns={"Retail Branding": "manufacturer"}, inplace=True)
result = pd.merge(result, 
                  devices[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')
result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,Model
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F


Работа с методом groupby

In [29]:
result.groupby("manufacturer").agg({
        "outgoing_mins_per_month": "mean",
        "outgoing_sms_per_month": "mean",
        "monthly_mb": "mean",
        "use_id": "count"
    })

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
manufacturer,,,,
HTC,299.842955,93.059318,5144.077955,44
Huawei,81.526667,9.500000,1561.226667,3
LGE,111.530000,12.760000,1557.330000,2
Lava,60.650000,261.900000,12458.670000,2
Lenovo,215.920000,12.930000,1557.330000,2
Motorola,95.127500,65.666250,3946.500000,16
OnePlus,354.855000,48.330000,6575.410000,6
Samsung,191.010093,92.390463,4017.318889,108
Sony,177.315625,40.176250,3212.000625,16


In [30]:
%%timeit
result.groupby("manufacturer").agg({
        "outgoing_mins_per_month": "mean",
        "outgoing_sms_per_month": "mean",
        "monthly_mb": "mean",
        "use_id": "count"
    })

100 loops, best of 3: 2.85 ms per loop


In [31]:
pysqldf("""SELECT manufacturer, AVG(outgoing_mins_per_month), AVG(outgoing_sms_per_month), AVG(monthly_mb), COUNT(use_id)
           FROM result
           GROUP BY manufacturer
        """)

,manufacturer,AVG(outgoing_mins_per_month),AVG(outgoing_sms_per_month),AVG(monthly_mb),COUNT(use_id)
0,None,289.751000,114.763000,3024.491000,10
1,HTC,299.842955,93.059318,5144.077955,44
2,Huawei,81.526667,9.500000,1561.226667,3
3,LGE,111.530000,12.760000,1557.330000,2
4,Lava,60.650000,261.900000,12458.670000,2
5,Lenovo,215.920000,12.930000,1557.330000,2
6,Motorola,95.127500,65.666250,3946.500000,16
7,OnePlus,354.855000,48.330000,6575.410000,6
8,Samsung,191.010093,92.390463,4017.318889,108
9,Sony,177.315625,40.176250,3212.000625,16


In [32]:
%%timeit
pysqldf("""SELECT manufacturer, AVG(outgoing_mins_per_month), AVG(outgoing_sms_per_month), AVG(monthly_mb), COUNT(use_id)
           FROM result
           GROUP BY manufacturer
        """)

100 loops, best of 3: 9.11 ms per loop
